# 🧪 Codelab: Grover's Search Algorithm

| Metadata | Value |
|----------|-------|
| **Algorithm** | Grover's Search (Amplitude Amplification) |
| **Difficulty** | 🟡 Intermediate |
| **Time** | 90-120 minutes |
| **Prerequisites** | Superposition, Phase kickback |
| **Qiskit Version** | 2.x |

---

## Learning Objectives

By the end of this codelab, you will be able to:

1. ✅ Build oracles for marking target states
2. ✅ Implement the Grover diffuser
3. ✅ Determine optimal iteration count
4. ✅ Handle multiple solutions
5. ✅ Visualize amplitude amplification

## Section 1: Environment Setup & Version Check

In [ ]:
# Required imports
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Optional, Tuple, Union
from math import floor, sqrt, pi

# Qiskit imports
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.quantum_info import Statevector, Operator
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit_aer import AerSimulator

# Version check - Qiskit 2.x required
import qiskit
version = qiskit.__version__
major_version = int(version.split('.')[0])
assert major_version >= 1, f"Qiskit 2.x required, found {version}"
print(f"✓ Qiskit version: {version}")
print(f"✓ NumPy version: {np.__version__}")
print("✓ All imports successful!")

## Section 2: Theory Recap

### The Problem

**Unstructured Search**: Find a marked item in a list of $N$ items with no structure to exploit.

- Classical: $O(N)$ queries (check each item)
- Quantum (Grover): $O(\sqrt{N})$ queries

### The Algorithm

1. **Initialize**: Start with uniform superposition $|s\rangle = H^{\otimes n}|0\rangle$
2. **Repeat** $\approx \frac{\pi}{4}\sqrt{N}$ times:
   - **Oracle**: Mark target with phase flip: $|w\rangle \to -|w\rangle$
   - **Diffuser**: Reflect about mean amplitude
3. **Measure**: Get the target with high probability

### Key Formulas

**Oracle**: $O_f|x\rangle = (-1)^{f(x)}|x\rangle$

**Diffuser**: $D_s = 2|s\rangle\langle s| - I = H^{\otimes n}(2|0\rangle\langle 0| - I)H^{\otimes n}$

**Optimal iterations**: $k = \lfloor \frac{\pi}{4}\sqrt{N/M} \rfloor$ where $M$ = number of targets

## Section 3: Building the Oracle

In [ ]:
def create_oracle(n_qubits: int, target: Union[int, str, List]) -> QuantumCircuit:
    """
    Create a phase oracle that marks target state(s) with a phase flip.
    
    Args:
        n_qubits: Number of qubits
        target: Target state(s) as integer, bitstring, or list of either
    
    Returns:
        Oracle circuit
    """
    qc = QuantumCircuit(n_qubits, name='Oracle')
    
    # Convert target to list if single value
    if not isinstance(target, list):
        target = [target]
    
    for t in target:
        # Convert to binary string if integer
        if isinstance(t, int):
            t = format(t, f'0{n_qubits}b')
        
        # Apply X gates to qubits that should be |0⟩ in target
        for i, bit in enumerate(reversed(t)):
            if bit == '0':
                qc.x(i)
        
        # Multi-controlled Z (marks |11...1⟩ after X transformations)
        if n_qubits == 1:
            qc.z(0)
        elif n_qubits == 2:
            qc.cz(0, 1)
        else:
            # Multi-controlled Z using decomposition
            qc.h(n_qubits - 1)
            qc.mcx(list(range(n_qubits - 1)), n_qubits - 1)
            qc.h(n_qubits - 1)
        
        # Undo X gates
        for i, bit in enumerate(reversed(t)):
            if bit == '0':
                qc.x(i)
    
    return qc


# Test oracle for 2-qubit case, target |11⟩
oracle_11 = create_oracle(2, '11')
print("Oracle for target |11⟩:")
print(oracle_11.draw())

# Verify oracle action
print("\nOracle verification:")
for state in ['00', '01', '10', '11']:
    qc = QuantumCircuit(2)
    # Prepare state
    for i, bit in enumerate(reversed(state)):
        if bit == '1':
            qc.x(i)
    qc.compose(oracle_11, inplace=True)
    
    sv = Statevector(qc)
    phase = np.angle(sv.data[int(state, 2)])
    sign = "+" if np.isclose(phase, 0) else "-"
    print(f"  |{state}⟩ → {sign}|{state}⟩")

In [ ]:
# Oracle for multiple targets
oracle_multi = create_oracle(3, ['000', '111'])
print("Oracle for targets |000⟩ and |111⟩:")
print(oracle_multi.draw())

# Verify
print("\nVerification:")
for i in range(8):
    state = format(i, '03b')
    qc = QuantumCircuit(3)
    for j, bit in enumerate(reversed(state)):
        if bit == '1':
            qc.x(j)
    qc.compose(oracle_multi, inplace=True)
    
    sv = Statevector(qc)
    phase = np.angle(sv.data[i])
    sign = "+" if np.isclose(phase, 0) else "-"
    marked = "← MARKED" if sign == "-" else ""
    print(f"  |{state}⟩ → {sign}|{state}⟩ {marked}")

## Section 4: Building the Diffuser

In [ ]:
def create_diffuser(n_qubits: int) -> QuantumCircuit:
    """
    Create the Grover diffuser (reflection about |s⟩).
    
    D_s = H^⊗n (2|0⟩⟨0| - I) H^⊗n
        = 2|s⟩⟨s| - I
    
    Args:
        n_qubits: Number of qubits
    
    Returns:
        Diffuser circuit
    """
    qc = QuantumCircuit(n_qubits, name='Diffuser')
    
    # Apply H to all qubits
    qc.h(range(n_qubits))
    
    # Apply X to all qubits
    qc.x(range(n_qubits))
    
    # Multi-controlled Z (marks |11...1⟩)
    if n_qubits == 1:
        qc.z(0)
    elif n_qubits == 2:
        qc.cz(0, 1)
    else:
        qc.h(n_qubits - 1)
        qc.mcx(list(range(n_qubits - 1)), n_qubits - 1)
        qc.h(n_qubits - 1)
    
    # Undo X gates
    qc.x(range(n_qubits))
    
    # Undo H gates
    qc.h(range(n_qubits))
    
    return qc


# Test diffuser
diffuser_2 = create_diffuser(2)
print("2-qubit Diffuser:")
print(diffuser_2.draw())

# Verify it's a reflection about |s⟩
print("\nDiffuser action on |s⟩ = (|00⟩+|01⟩+|10⟩+|11⟩)/2:")
qc = QuantumCircuit(2)
qc.h([0, 1])  # Create |s⟩
sv_before = Statevector(qc)
print(f"  Before: {sv_before.data.round(4)}")

qc.compose(diffuser_2, inplace=True)
sv_after = Statevector(qc)
print(f"  After:  {sv_after.data.round(4)}")
print("  (|s⟩ is unchanged by reflection about |s⟩!)")

In [ ]:
# Visualize the diffuser as "reflection about mean"
def demonstrate_reflection(amplitudes: np.ndarray) -> np.ndarray:
    """
    Apply reflection about mean to amplitudes.
    """
    mean = np.mean(amplitudes)
    reflected = 2 * mean - amplitudes
    return reflected


# Example: After oracle marks one state
initial = np.array([0.5, 0.5, 0.5, 0.5])  # |s⟩
after_oracle = np.array([0.5, 0.5, 0.5, -0.5])  # Oracle flipped |11⟩
after_diffuser = demonstrate_reflection(after_oracle)

print("Reflection About Mean Demo:")
print(f"  After oracle:   {after_oracle}")
print(f"  Mean amplitude: {np.mean(after_oracle):.4f}")
print(f"  After diffuser: {after_diffuser}")
print(f"\n  |11⟩ amplitude went from {after_oracle[3]:.2f} to {after_diffuser[3]:.2f}!")

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(12, 4))
states = ['|00⟩', '|01⟩', '|10⟩', '|11⟩']

for ax, (title, amps) in zip(axes, [('Initial |s⟩', initial), 
                                      ('After Oracle', after_oracle),
                                      ('After Diffuser', after_diffuser)]):
    colors = ['green' if a > 0 else 'red' for a in amps]
    ax.bar(states, amps, color=colors, alpha=0.7)
    ax.axhline(y=0, color='black', linewidth=0.5)
    ax.axhline(y=np.mean(amps), color='blue', linestyle='--', label='mean')
    ax.set_ylim(-0.6, 1.1)
    ax.set_ylabel('Amplitude')
    ax.set_title(title)
    ax.legend()

plt.tight_layout()
plt.show()

## Section 5: Complete Grover's Algorithm

In [ ]:
def grover_circuit(n_qubits: int, target: Union[int, str, List], 
                   iterations: Optional[int] = None) -> QuantumCircuit:
    """
    Build complete Grover's algorithm circuit.
    
    Args:
        n_qubits: Number of qubits
        target: Target state(s)
        iterations: Number of Grover iterations (default: optimal)
    
    Returns:
        Complete Grover circuit with measurements
    """
    # Determine number of targets
    if isinstance(target, list):
        M = len(target)
    else:
        M = 1
    
    N = 2 ** n_qubits
    
    # Optimal iterations
    if iterations is None:
        iterations = max(1, floor(pi / 4 * sqrt(N / M)))
    
    # Build circuit
    qc = QuantumCircuit(n_qubits, n_qubits)
    
    # Initialize superposition
    qc.h(range(n_qubits))
    
    qc.barrier(label='init')
    
    # Grover iterations
    oracle = create_oracle(n_qubits, target)
    diffuser = create_diffuser(n_qubits)
    
    for i in range(iterations):
        qc.compose(oracle, inplace=True)
        qc.compose(diffuser, inplace=True)
        qc.barrier(label=f'G{i+1}')
    
    # Measure
    qc.measure(range(n_qubits), range(n_qubits))
    
    return qc


# Build Grover circuit for 2 qubits, target |11⟩
grover_2q = grover_circuit(2, '11')
print("Grover's Algorithm (2 qubits, target |11⟩):")
print(f"Optimal iterations: {max(1, floor(pi/4 * sqrt(4)))}")
print(grover_2q.draw())

In [ ]:
def run_grover(n_qubits: int, target: Union[int, str, List],
               iterations: Optional[int] = None, shots: int = 1024) -> dict:
    """
    Run Grover's algorithm and return results.
    """
    qc = grover_circuit(n_qubits, target, iterations)
    
    simulator = AerSimulator()
    result = simulator.run(qc, shots=shots).result()
    counts = result.get_counts()
    
    # Normalize target format
    if isinstance(target, list):
        targets = [t if isinstance(t, str) else format(t, f'0{n_qubits}b') 
                   for t in target]
    else:
        targets = [target if isinstance(target, str) else format(target, f'0{n_qubits}b')]
    
    # Calculate success rate
    # Note: Qiskit returns results in LSB order
    success_count = sum(counts.get(t[::-1], 0) for t in targets)
    success_rate = success_count / shots
    
    return {
        'counts': counts,
        'success_rate': success_rate,
        'targets': targets,
        'iterations': iterations or max(1, floor(pi/4 * sqrt(2**n_qubits / len(targets))))
    }


# Run Grover for 2 qubits
results_2q = run_grover(2, '11', shots=1024)
print("Results (2 qubits, target |11⟩):")
print(f"  Iterations: {results_2q['iterations']}")
print(f"  Success rate: {results_2q['success_rate']:.1%}")
print(f"  Counts: {results_2q['counts']}")

In [ ]:
# Test with larger circuit
print("Grover's Algorithm Performance")
print("=" * 50)

for n in range(2, 6):
    N = 2 ** n
    target = N // 2  # Pick middle target
    
    results = run_grover(n, target, shots=1024)
    
    classical_queries = N // 2  # Average for classical
    quantum_queries = results['iterations']  # Grover iterations
    
    print(f"\n{n} qubits (N={N}):")
    print(f"  Target: |{format(target, f'0{n}b')}⟩")
    print(f"  Grover iterations: {quantum_queries}")
    print(f"  Classical queries (avg): {classical_queries}")
    print(f"  Speedup: {classical_queries / quantum_queries:.1f}x")
    print(f"  Success rate: {results['success_rate']:.1%}")

## Section 5.5: State Evolution Analysis

### 5.5.1 The Grover Operator and the "Amplitude Amplification via Reflection" Rule

**The Core Identity:**
The Grover operator $G$ consists of two reflections:

$$G = D \cdot O = (2|s\rangle\langle s| - I)(I - 2|w\rangle\langle w|)$$

where:
- $O = I - 2|w\rangle\langle w|$ is the **oracle** (reflects about $|w\rangle$)
- $D = 2|s\rangle\langle s| - I$ is the **diffuser** (reflects about $|s\rangle$)
- $|s\rangle = H^{\otimes n}|0\rangle$ is the uniform superposition
- $|w\rangle$ is the winner/target state

**The Amplitude Amplification Rule:**
From L3.6: *"The nice thing this does is this new state now is closer to the winner state. It has a higher chance to fall into the winner state if I do a measurement now. And that is the amplitude amplification trick."*

**Key Geometric Insight:**
Two successive reflections = one rotation! The angle of rotation is $2\theta$ where:
$$\sin(\theta) = \frac{1}{\sqrt{N}}$$

After $k$ iterations, the state makes angle $(2k+1)\theta$ with the non-target subspace.

In [ ]:
def demonstrate_amplitude_amplification(N: int, k_iterations: int = None):
    """
    Demonstrate the amplitude amplification principle.
    
    From L3.6: "Initially, the probability of observing winner was 1/N.
    By utilizing this amplification of the amplitude for my winner by 
    taking advantage of the Oracle, this amplitude can be enhanced to 
    value greater than half."
    """
    import numpy as np
    
    # Initial angle
    theta = np.arcsin(1 / np.sqrt(N))
    
    # Optimal number of iterations
    k_opt = int(np.round(np.pi / (4 * theta) - 0.5))
    
    if k_iterations is None:
        k_iterations = k_opt
    
    print("Amplitude Amplification Demonstration")
    print("=" * 60)
    print(f"Database size: N = {N}")
    print(f"Initial angle: θ = arcsin(1/√N) = {np.degrees(theta):.2f}°")
    print(f"Optimal iterations: k_opt = ⌊π/(4θ)⌋ = {k_opt}")
    print()
    
    print("Evolution of target probability:")
    print("-" * 60)
    print(f"{'Iteration k':<12} {'Angle':<20} {'P(winner)':<15} {'Status'}")
    print("-" * 60)
    
    for k in range(k_iterations + 2):
        angle = (2*k + 1) * theta
        prob = np.sin(angle) ** 2
        
        status = ""
        if k == 0:
            status = "← Initial (1/N)"
        elif k == k_opt:
            status = "← OPTIMAL"
        elif prob < 0.5:
            status = "← Amplifying..."
        elif prob > 0.99:
            status = "← Near unity!"
        elif k > k_opt:
            status = "← Over-rotating!"
        
        print(f"{k:<12} {np.degrees(angle):>8.2f}° ({angle/np.pi:.3f}π)  "
              f"{prob:>8.4f}  {status}")
    
    print()
    print("Key insight: P(winner) = sin²((2k+1)θ)")
    print(f"At k_opt = {k_opt}: angle ≈ π/2, so sin² ≈ 1")
    
    return theta, k_opt


# Demonstrate for N = 16 (4 qubits)
theta, k_opt = demonstrate_amplitude_amplification(N=16)

### 5.5.2 State Evolution: The 2D Geometry

From L3.6: *"This S can be written as my winner state plus all other possible states... If you subtract from it the winner state, I'm denoting by S prime... this angle is 90 degrees because all of these states are orthogonal."*

The entire evolution happens in a **2D subspace** spanned by:
- $|w\rangle$ - the target/winner state  
- $|s'\rangle$ - the uniform superposition over all non-target states

| Stage | State | Angle from $\|s'\rangle$ | $P(\|w\rangle)$ |
|-------|-------|--------------------------|-----------------|
| Initial | $\|s\rangle = H^{\otimes n}\|0\rangle$ | $\theta$ | $\sin^2\theta = 1/N$ |
| After Oracle | $O\|s\rangle$ | $-\theta$ | $1/N$ (same!) |
| After Diffuser | $DO\|s\rangle$ | $3\theta$ | $\sin^2(3\theta)$ |
| After $k$ iterations | $G^k\|s\rangle$ | $(2k+1)\theta$ | $\sin^2((2k+1)\theta)$ |
| Optimal $k^*$ | $G^{k^*}\|s\rangle$ | $\approx\pi/2$ | $\approx 1$ |

**Why the oracle alone doesn't help:**  
The oracle flips the sign but doesn't change probability! The diffuser is essential.

In [ ]:
def trace_grover_evolution(n_qubits: int, target: str, max_iterations: int = None, verbose: bool = True) -> dict:
    """
    Trace the quantum state evolution through Grover's algorithm.
    
    From L3.6: "The result matrix... can also be written as 2|s⟩⟨s| - I.
    If I apply this operator on any arbitrary state psi, it will end up 
    just reflecting that above the state S."
    """
    import numpy as np
    
    N = 2 ** n_qubits
    target_idx = int(target, 2)
    theta = np.arcsin(1 / np.sqrt(N))
    
    if max_iterations is None:
        max_iterations = int(np.round(np.pi / (4 * theta)))
    
    stages = {}
    
    if verbose:
        print("Grover's Algorithm State Evolution Trace")
        print("=" * 70)
        print(f"Parameters: n={n_qubits} qubits, N={N} states")
        print(f"Target: |{target}⟩ (index {target_idx})")
        print(f"Initial angle: θ = {np.degrees(theta):.2f}°")
        print()
    
    # Stage 0: After Hadamards (uniform superposition)
    if verbose:
        print("Stage 0 - After Hadamards (uniform superposition):")
        print("-" * 70)
        print(f"  |s⟩ = H⊗ⁿ|0⟩ = (1/√{N}) Σ|x⟩")
        print(f"  All amplitudes: 1/√{N} = {1/np.sqrt(N):.4f}")
        print(f"  P(target) = 1/N = {1/N:.6f}")
        print(f"  Angle from |s'⟩: θ = {np.degrees(theta):.2f}°")
        print()
    
    # Detailed iteration trace
    if verbose:
        print("Iteration-by-Iteration Evolution:")
        print("-" * 70)
    
    for k in range(max_iterations + 1):
        angle_before = (2*k + 1) * theta - 2*theta if k > 0 else theta
        
        if k > 0:
            # After oracle (reflection about |w⟩)
            angle_after_oracle = -angle_before + 2*theta if k == 1 else (2*k - 1) * theta
            
            if verbose and k <= 3:
                print(f"  Iteration {k}:")
                print(f"    Oracle: Reflect about |w⟩ (flip sign of |w⟩ component)")
                print(f"      Angle: {np.degrees(angle_before):.2f}° → reflected")
            
            # After diffuser (reflection about |s⟩)
            angle_after_diffuser = (2*k + 1) * theta
            prob_after = np.sin(angle_after_diffuser) ** 2
            
            if verbose and k <= 3:
                print(f"    Diffuser: Reflect about |s⟩")
                print(f"      Angle: → {np.degrees(angle_after_diffuser):.2f}°")
                print(f"      P(target) = sin²({2*k+1}θ) = {prob_after:.4f}")
                print()
        else:
            prob_after = np.sin(theta) ** 2
            if verbose:
                print(f"  k=0 (initial): angle = θ = {np.degrees(theta):.2f}°, P(target) = {prob_after:.6f}")
                print()
    
    # Final summary
    k_opt = int(np.round(np.pi / (4 * theta) - 0.5))
    final_angle = (2*k_opt + 1) * theta
    final_prob = np.sin(final_angle) ** 2
    
    if verbose:
        print("Optimal Point:")
        print("-" * 70)
        print(f"  k* = ⌊π/(4θ) - 1/2⌋ = {k_opt}")
        print(f"  Final angle: (2·{k_opt}+1)·θ = {np.degrees(final_angle):.2f}° ≈ 90°")
        print(f"  P(target) = {final_prob:.6f}")
        print()
        print("Why √N iterations?")
        print(f"  Rotation per iteration: 2θ ≈ 2/√N radians")
        print(f"  Total rotation needed: ≈ π/2 radians")
        print(f"  Iterations needed: (π/2)/(2/√N) = (π/4)√N ≈ {np.pi/4 * np.sqrt(N):.1f}")
    
    stages['theta'] = theta
    stages['k_opt'] = k_opt
    stages['final_prob'] = final_prob
    
    return stages


# Trace evolution for 3 qubits
stages = trace_grover_evolution(n_qubits=3, target='101', max_iterations=3)

### 5.5.3 The Diffuser: Why Hadamard Sandwiches Work

From L3.6: *"The V I can do this coordinate transform... The transformation that maps this uniform state to state |0...0⟩ is nothing but the Hadamard. So then I will have H applied, then this gate, then H again."*

**The Diffuser Circuit:**
$$D = H^{\otimes n} (2|0\rangle\langle 0| - I) H^{\otimes n} = 2|s\rangle\langle s| - I$$

**Why this works:**
1. **Hadamard maps** $|s\rangle \leftrightarrow |0...0\rangle$
2. In the $|0\rangle$ basis, flipping about $|0...0\rangle$ is easy: negate all except first element
3. **Transform back** with Hadamard

**The Matrix View:**
From L3.6: *"This matrix over here would flip any given vector about the state |0,0,0⟩. It leaves the first element unchanged, but puts a minus sign in front of all other elements."*

$$2|0\rangle\langle 0| - I = \begin{pmatrix} 1 & 0 & \cdots \\ 0 & -1 & \cdots \\ \vdots & & \ddots \end{pmatrix}$$

In [ ]:
def explore_reflection_geometry(N: int = 8, k_max: int = 5):
    """
    Visualize the 2D reflection geometry of Grover's algorithm.
    
    From L3.6: "After one cycle of doing amplitude amplification, 
    I rotated this uniform superposition state by an angle 2θ."
    """
    import numpy as np
    import matplotlib.pyplot as plt
    
    theta = np.arcsin(1 / np.sqrt(N))
    k_opt = int(np.round(np.pi / (4 * theta) - 0.5))
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Left plot: Geometric picture
    ax1 = axes[0]
    
    # Draw axes
    ax1.axhline(y=0, color='gray', linestyle='-', alpha=0.3)
    ax1.axvline(x=0, color='gray', linestyle='-', alpha=0.3)
    
    # Draw |s'⟩ (x-axis) and |w⟩ (y-axis)
    ax1.annotate('', xy=(1.1, 0), xytext=(0, 0),
                arrowprops=dict(arrowstyle='->', color='black', lw=2))
    ax1.annotate('', xy=(0, 1.1), xytext=(0, 0),
                arrowprops=dict(arrowstyle='->', color='black', lw=2))
    ax1.text(1.15, 0, "|s'⟩", fontsize=12, ha='left', va='center')
    ax1.text(0, 1.15, "|w⟩", fontsize=12, ha='center', va='bottom')
    
    # Draw state vectors for each iteration
    colors = plt.cm.viridis(np.linspace(0, 0.8, k_max + 1))
    
    for k in range(min(k_max + 1, k_opt + 2)):
        angle = (2*k + 1) * theta
        x = np.cos(angle)
        y = np.sin(angle)
        
        ax1.annotate('', xy=(x, y), xytext=(0, 0),
                    arrowprops=dict(arrowstyle='->', color=colors[k], lw=2))
        
        label = f'k={k}' if k < k_opt else f'k={k} (opt)' if k == k_opt else f'k={k}'
        ax1.text(x*1.1, y*1.1, label, fontsize=10, ha='center', color=colors[k])
    
    # Draw angle arc for initial theta
    arc_angles = np.linspace(0, theta, 20)
    arc_r = 0.3
    ax1.plot(arc_r * np.cos(arc_angles), arc_r * np.sin(arc_angles), 'b-', lw=1.5)
    ax1.text(arc_r * 1.3 * np.cos(theta/2), arc_r * 1.3 * np.sin(theta/2), 'θ', 
             fontsize=12, color='blue')
    
    ax1.set_xlim(-0.2, 1.3)
    ax1.set_ylim(-0.2, 1.3)
    ax1.set_aspect('equal')
    ax1.set_title(f'Grover Rotation in 2D Subspace (N={N})')
    ax1.grid(True, alpha=0.3)
    
    # Right plot: Probability vs iterations
    ax2 = axes[1]
    
    iterations = np.arange(0, k_max + 1)
    angles = (2 * iterations + 1) * theta
    probs = np.sin(angles) ** 2
    
    ax2.bar(iterations, probs, color=colors[:len(iterations)], edgecolor='black', alpha=0.7)
    ax2.axhline(y=1/N, color='red', linestyle='--', label=f'Initial: 1/N = {1/N:.4f}')
    ax2.axvline(x=k_opt, color='green', linestyle='--', label=f'Optimal k* = {k_opt}')
    
    ax2.set_xlabel('Iterations (k)')
    ax2.set_ylabel('P(target)')
    ax2.set_title('Probability Amplification')
    ax2.set_ylim(0, 1.1)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nKey Observations:")
    print(f"  • Initial angle θ = arcsin(1/√{N}) = {np.degrees(theta):.2f}°")
    print(f"  • Each Grover iteration rotates by 2θ = {np.degrees(2*theta):.2f}°")
    print(f"  • Optimal k* = {k_opt} gets angle to ≈ 90°")
    print(f"  • Beyond k*, probability DECREASES (over-rotation)")


# Visualize for N=8 (3 qubits)
explore_reflection_geometry(N=8, k_max=5)

### 5.5.4 Interactive: Multiple Targets and Over-Rotation

**Multiple Marked Items:**
When there are $M$ targets instead of 1, the rotation angle changes:
$$\sin(\theta) = \sqrt{\frac{M}{N}}$$

This means:
- **Faster amplification** (larger angle per iteration)
- **Fewer iterations needed**: $k^* \approx \frac{\pi}{4}\sqrt{N/M}$
- **Risk of over-rotation** increases!

**The Over-Rotation Problem:**
From L3.6: *"The more accuracy you want, you can keep on repeating this process more and more and get to a higher and higher accuracy."*

⚠️ **But this is WRONG for Grover!** Unlike classical search, more iterations can make things worse!

In [ ]:
def explore_grover_variants(N: int = 64, M_values: list = [1, 2, 4, 8], max_k: int = 15):
    """
    Explore Grover's algorithm with different numbers of marked items.
    
    From L3.7: "To get an amplitude value of unity, which implies probability equals
    to one, we need square root N set of reflections."
    """
    import numpy as np
    import matplotlib.pyplot as plt
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Left: Probability vs iterations for different M
    ax1 = axes[0]
    
    for M in M_values:
        theta = np.arcsin(np.sqrt(M / N))
        k_opt = int(np.round(np.pi / (4 * theta) - 0.5))
        
        iterations = np.arange(0, max_k + 1)
        angles = (2 * iterations + 1) * theta
        probs = np.sin(angles) ** 2
        
        ax1.plot(iterations, probs, 'o-', label=f'M={M} (k*={k_opt})', markersize=4)
        ax1.axvline(x=k_opt, linestyle=':', alpha=0.5)
    
    ax1.set_xlabel('Iterations (k)')
    ax1.set_ylabel('P(target)')
    ax1.set_title(f'Effect of Number of Marked Items (N={N})')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(0, 1.1)
    
    # Right: Optimal iterations vs M
    ax2 = axes[1]
    
    M_range = np.arange(1, N // 2 + 1)
    k_opts = []
    
    for M in M_range:
        theta = np.arcsin(np.sqrt(M / N))
        k_opt = max(1, int(np.round(np.pi / (4 * theta) - 0.5)))
        k_opts.append(k_opt)
    
    ax2.plot(M_range, k_opts, 'b-', lw=2)
    ax2.plot(M_range, np.pi/4 * np.sqrt(N / M_range), 'r--', lw=1, label='π/4 · √(N/M)')
    
    ax2.set_xlabel('Number of marked items (M)')
    ax2.set_ylabel('Optimal iterations (k*)')
    ax2.set_title('Optimal Iterations vs Marked Items')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\nKey Insights:")
    print(f"  • More targets M → larger rotation angle θ")
    print(f"  • Optimal iterations k* ∝ √(N/M)")
    print(f"  • When M = N/4, k* = 1 (one iteration suffices!)")
    print(f"  • Over-rotation causes oscillation back to low probability")


# Explore with N=64
explore_grover_variants(N=64, M_values=[1, 4, 8, 16])

In [ ]:
def demonstrate_over_rotation(n_qubits: int = 4, target: str = '1010'):
    """
    Demonstrate the over-rotation problem in Grover's algorithm.
    """
    import numpy as np
    
    N = 2 ** n_qubits
    theta = np.arcsin(1 / np.sqrt(N))
    k_opt = int(np.round(np.pi / (4 * theta) - 0.5))
    
    print("Over-Rotation Demonstration")
    print("=" * 60)
    print(f"N = {N}, θ = {np.degrees(theta):.2f}°, k* = {k_opt}")
    print()
    
    print(f"{'Iterations':<12} {'Total Angle':<15} {'P(target)':<12} {'Status'}")
    print("-" * 60)
    
    for k in range(2 * k_opt + 3):
        angle = (2*k + 1) * theta
        prob = np.sin(angle) ** 2
        
        if k < k_opt:
            status = "↗ Amplifying"
        elif k == k_opt:
            status = "★ OPTIMAL"
        elif prob > 0.5:
            status = "↘ Over-rotating (still ok)"
        else:
            status = "⚠ Over-rotated (< 50%)"
        
        angle_deg = np.degrees(angle)
        angle_normalized = (angle_deg % 180)
        print(f"{k:<12} {angle_deg:>6.1f}° ({angle_normalized:>5.1f}° mod 180°)  "
              f"{prob:>8.4f}   {status}")
    
    print()
    print("⚠️  Warning: Grover is NOT monotonic!")
    print("    Unlike classical search, MORE iterations can give WORSE results.")
    print("    You must know (approximately) when to stop.")


# Show over-rotation for 4 qubits
demonstrate_over_rotation(n_qubits=4)

### 5.5.5 Key Takeaways: State Evolution Summary

| Aspect | Grover's Algorithm |
|--------|-------------------|
| **Core Identity** | $G = (2\|s\rangle\langle s\| - I)(I - 2\|w\rangle\langle w\|)$ |
| **Geometric Picture** | Two reflections = rotation by $2\theta$ in 2D subspace |
| **Initial Angle** | $\sin\theta = 1/\sqrt{N}$ (very small for large N) |
| **State After $k$ Iterations** | Angle = $(2k+1)\theta$ from $\|s'\rangle$ |
| **Success Probability** | $P(w) = \sin^2((2k+1)\theta)$ |
| **Optimal Iterations** | $k^* = \lfloor\frac{\pi}{4}\sqrt{N}\rfloor$ |
| **Quantum Advantage** | $O(\sqrt{N})$ vs $O(N)$ classical queries |
| **Key Limitation** | Over-rotation if $k > k^*$ |

**Why It Works:**
1. **Uniform superposition** $\|s\rangle$ has tiny overlap $1/\sqrt{N}$ with target
2. **Oracle** reflects about target, flipping its amplitude (but not probability!)
3. **Diffuser** reflects about mean, amplifying target while suppressing others
4. **Combined rotation** moves state toward target by $2\theta$ per iteration
5. After $\approx\frac{\pi}{4}\sqrt{N}$ iterations, angle reaches $\approx 90°$ → P ≈ 1

**The "Magic" of Grover:**
From L3.6: *"Initially, the probability of observing winner was 1/N... By utilizing this amplification, this amplitude can be enhanced to value greater than half."*

**Why $\sqrt{N}$ is Optimal (not $\log N$!):**
- Each iteration rotates by $2\theta \approx 2/\sqrt{N}$
- Need total rotation of $\pi/2$
- Iterations needed: $(\pi/2)/(2/\sqrt{N}) = \frac{\pi}{4}\sqrt{N}$
- This is provably optimal for unstructured search (BBBV theorem)

## Section 6: Visualizing Amplitude Amplification

In [ ]:
def visualize_grover_evolution(n_qubits: int, target: str, max_iterations: int = 10):
    """
    Visualize how amplitudes evolve during Grover iterations.
    """
    N = 2 ** n_qubits
    target_idx = int(target, 2)
    
    # Track target and non-target amplitudes
    target_probs = []
    
    for k in range(max_iterations + 1):
        # Build circuit with k iterations
        qc = QuantumCircuit(n_qubits)
        qc.h(range(n_qubits))
        
        oracle = create_oracle(n_qubits, target)
        diffuser = create_diffuser(n_qubits)
        
        for _ in range(k):
            qc.compose(oracle, inplace=True)
            qc.compose(diffuser, inplace=True)
        
        # Get statevector
        sv = Statevector(qc)
        probs = np.abs(sv.data) ** 2
        
        target_probs.append(probs[target_idx])
    
    # Plot
    fig, ax = plt.subplots(figsize=(10, 5))
    
    iterations = range(max_iterations + 1)
    ax.plot(iterations, target_probs, 'o-', linewidth=2, markersize=8, label=f'P(|{target}⟩)')
    
    # Mark optimal
    k_opt = max(1, floor(pi / 4 * sqrt(N)))
    ax.axvline(x=k_opt, color='red', linestyle='--', label=f'Optimal k={k_opt}')
    
    ax.set_xlabel('Iterations')
    ax.set_ylabel('Probability of target')
    ax.set_title(f'Grover Amplitude Amplification ({n_qubits} qubits, target |{target}⟩)')
    ax.set_ylim(0, 1.1)
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return target_probs


# Visualize for 4 qubits
probs = visualize_grover_evolution(4, '1010', max_iterations=8)

print("\n💡 Observation: Probability OSCILLATES!")
print(f"   Too few iterations: low probability")
print(f"   Optimal iterations: maximum probability (~1)")
print(f"   Too many iterations: probability decreases again")

In [ ]:
# Geometric visualization
def geometric_visualization(n_qubits: int, target: str, iterations: int = 4):
    """
    Visualize Grover's algorithm in the 2D plane spanned by |s'⟩ and |w⟩.
    """
    N = 2 ** n_qubits
    
    # Initial angle
    theta = np.arcsin(1 / np.sqrt(N))
    
    fig, ax = plt.subplots(figsize=(8, 8))
    
    # Draw axes
    ax.arrow(0, 0, 1.2, 0, head_width=0.03, head_length=0.03, fc='gray', ec='gray')
    ax.arrow(0, 0, 0, 1.2, head_width=0.03, head_length=0.03, fc='gray', ec='gray')
    ax.text(1.25, 0, "|s'⟩", fontsize=12)
    ax.text(0, 1.25, "|w⟩", fontsize=12)
    
    # Draw states after each iteration
    colors = plt.cm.viridis(np.linspace(0, 1, iterations + 1))
    
    for k in range(iterations + 1):
        angle = (2 * k + 1) * theta
        x = np.cos(angle)
        y = np.sin(angle)
        
        ax.arrow(0, 0, x * 0.95, y * 0.95, head_width=0.03, head_length=0.03, 
                 fc=colors[k], ec=colors[k], linewidth=2)
        ax.plot(x, y, 'o', color=colors[k], markersize=10)
        ax.annotate(f'G^{k}|s⟩', (x + 0.05, y + 0.05), fontsize=10, color=colors[k])
    
    # Draw unit circle arc
    angles = np.linspace(0, np.pi/2, 100)
    ax.plot(np.cos(angles), np.sin(angles), 'k--', alpha=0.3)
    
    # Mark initial angle
    ax.annotate(f'θ = {np.degrees(theta):.1f}°', 
                (0.4, 0.1), fontsize=10)
    
    ax.set_xlim(-0.2, 1.4)
    ax.set_ylim(-0.2, 1.4)
    ax.set_aspect('equal')
    ax.set_title(f'Geometric View: Rotation by 2θ per iteration\n({n_qubits} qubits, N={N})')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


geometric_visualization(4, '1010', iterations=4)

## Section 7: Multiple Solutions

In [ ]:
def grover_multiple_targets(n_qubits: int, targets: List[str], shots: int = 2048):
    """
    Test Grover with multiple target states.
    """
    N = 2 ** n_qubits
    M = len(targets)
    
    k_opt = max(1, floor(pi / 4 * sqrt(N / M)))
    
    print(f"Multiple Target Grover Search")
    print(f"="*50)
    print(f"N = {N}, M = {M} targets")
    print(f"Targets: {targets}")
    print(f"Optimal iterations: ⌊(π/4)√(N/M)⌋ = ⌊(π/4)√({N}/{M})⌋ = {k_opt}")
    
    results = run_grover(n_qubits, targets, iterations=k_opt, shots=shots)
    
    print(f"\nResults:")
    print(f"  Success rate: {results['success_rate']:.1%}")
    print(f"  Counts (top 5):")
    
    sorted_counts = sorted(results['counts'].items(), key=lambda x: -x[1])
    for bitstring, count in sorted_counts[:5]:
        # Convert to correct bit order for display
        state = bitstring[::-1]
        is_target = "← TARGET" if state in targets else ""
        print(f"    |{state}⟩: {count} ({count/shots:.1%}) {is_target}")
    
    return results


# Test with 2 targets out of 16 states
results_multi = grover_multiple_targets(4, ['0000', '1111'], shots=2048)

In [ ]:
# Effect of M on iterations
print("\nEffect of Number of Targets on Optimal Iterations")
print("=" * 60)

n = 6
N = 2 ** n

print(f"\n{n} qubits (N = {N}):")
print(f"{'M targets':<12} | {'k_opt':<8} | {'Expected success':<18}")
print("-" * 60)

for M in [1, 2, 4, 8, 16, 32]:
    theta = np.arcsin(np.sqrt(M / N))
    k_opt = max(1, floor(pi / 4 / theta))
    success_prob = np.sin((2 * k_opt + 1) * theta) ** 2
    
    print(f"{M:<12} | {k_opt:<8} | {success_prob:.1%}")

## Section 8: Trap Demonstrations

In [ ]:
# TRAP 1: Too many iterations (overshooting)
print("TRAP 1: Overshooting with Too Many Iterations")
print("=" * 50)

n = 4
N = 2 ** n
k_opt = max(1, floor(pi / 4 * sqrt(N)))

print(f"Search space: N = {N}")
print(f"Optimal iterations: {k_opt}")
print(f"\n{'Iterations':<12} | {'Success Rate':<15}")
print("-" * 35)

for k in [1, k_opt, k_opt * 2, k_opt * 3, k_opt * 4]:
    results = run_grover(n, '1010', iterations=k, shots=1024)
    status = "← OPTIMAL" if k == k_opt else ""
    print(f"{k:<12} | {results['success_rate']:.1%} {status}")

print("\n⚠️  Success probability OSCILLATES with iterations!")

In [ ]:
# TRAP 2: Oracle that flips bits instead of phases
print("\nTRAP 2: Wrong Oracle (Bit Flip vs Phase Flip)")
print("=" * 50)

def wrong_oracle(n_qubits: int, target: str) -> QuantumCircuit:
    """BUGGY: Flips bits instead of phase."""
    qc = QuantumCircuit(n_qubits, name='WrongOracle')
    
    # This just applies X gates - doesn't mark with phase!
    for i, bit in enumerate(reversed(target)):
        if bit == '1':
            qc.x(i)  # BUG: This flips the bit, not the phase
    
    return qc


# Compare
n = 3
target = '101'

# Correct oracle
qc_correct = QuantumCircuit(n, n)
qc_correct.h(range(n))
qc_correct.compose(create_oracle(n, target), inplace=True)
qc_correct.compose(create_diffuser(n), inplace=True)
qc_correct.measure(range(n), range(n))

# Wrong oracle
qc_wrong = QuantumCircuit(n, n)
qc_wrong.h(range(n))
qc_wrong.compose(wrong_oracle(n, target), inplace=True)
qc_wrong.compose(create_diffuser(n), inplace=True)
qc_wrong.measure(range(n), range(n))

# Run both
sim = AerSimulator()
counts_correct = sim.run(qc_correct, shots=1024).result().get_counts()
counts_wrong = sim.run(qc_wrong, shots=1024).result().get_counts()

print(f"Target: |{target}⟩")
print(f"\nCorrect oracle (phase flip):")
for bs, c in sorted(counts_correct.items(), key=lambda x: -x[1])[:3]:
    print(f"  |{bs[::-1]}⟩: {c} ({c/1024:.1%})")

print(f"\nWrong oracle (bit flip):")
for bs, c in sorted(counts_wrong.items(), key=lambda x: -x[1])[:3]:
    print(f"  |{bs[::-1]}⟩: {c} ({c/1024:.1%})")

print("\n⚠️  Bit flip oracle doesn't amplify the target!")

In [ ]:
# TRAP 3: Forgetting initial superposition
print("\nTRAP 3: Missing Initial Superposition")
print("=" * 50)

n = 3
target = '101'

# Correct: with H gates
qc_correct = QuantumCircuit(n, n)
qc_correct.h(range(n))  # ← Important!
qc_correct.compose(create_oracle(n, target), inplace=True)
qc_correct.compose(create_diffuser(n), inplace=True)
qc_correct.measure(range(n), range(n))

# Wrong: without H gates
qc_wrong = QuantumCircuit(n, n)
# Missing H gates!
qc_wrong.compose(create_oracle(n, target), inplace=True)
qc_wrong.compose(create_diffuser(n), inplace=True)
qc_wrong.measure(range(n), range(n))

# Run both
counts_correct = sim.run(qc_correct, shots=1024).result().get_counts()
counts_wrong = sim.run(qc_wrong, shots=1024).result().get_counts()

print(f"Target: |{target}⟩")
print(f"\nWith initial H gates:")
for bs, c in sorted(counts_correct.items(), key=lambda x: -x[1])[:2]:
    print(f"  |{bs[::-1]}⟩: {c} ({c/1024:.1%})")

print(f"\nWithout initial H gates:")
for bs, c in sorted(counts_wrong.items(), key=lambda x: -x[1])[:2]:
    print(f"  |{bs[::-1]}⟩: {c} ({c/1024:.1%})")

print("\n⚠️  Without superposition, algorithm doesn't work!")

## Section 9: Exercises

### Exercise 1: Custom Oracle (Beginner)
Implement Grover's search for finding states where the first and last qubits are equal (|00x...x0⟩ or |11x...x1⟩) in a 4-qubit system.

In [ ]:
# TODO: Exercise 1
# 1. Identify all 4-bit strings where first bit == last bit
# 2. Build oracle for these targets
# 3. Run Grover and verify

# Your code here:

### Exercise 2: Iteration Analysis (Intermediate)
Plot the success probability vs. iterations for N=64 (6 qubits) and verify that the probability follows $\sin^2((2k+1)\theta)$.

In [ ]:
# TODO: Exercise 2
# Your code here:

### Exercise 3: SAT Solving (Advanced)
Use Grover's algorithm to solve the SAT problem: Find x₁, x₂, x₃ such that (x₁ OR x₂) AND (NOT x₂ OR x₃) AND (x₁ OR NOT x₃) is TRUE.

In [ ]:
# TODO: Exercise 3
# 1. Build an oracle that marks satisfying assignments
# 2. Run Grover's algorithm
# 3. Verify the solution

# Your code here:

## Section 10: Quick Knowledge Check

**Q1**: Why is the quadratic speedup optimal for unstructured search?

<details>
<summary>Click for answer</summary>

There's a formal proof (Bennett, Bernstein, Brassard, Vazirani 1997) showing that any quantum algorithm for unstructured search requires $\Omega(\sqrt{N})$ queries. This makes Grover's algorithm provably optimal!

The intuition: quantum amplitude can only "spread" at a certain rate, and extracting information about a marked item requires accumulating enough amplitude, which takes $\sqrt{N}$ steps.
</details>

**Q2**: What happens if we don't know how many solutions exist?

<details>
<summary>Click for answer</summary>

If we don't know M:
1. **Quantum Counting**: Use QPE on the Grover operator to estimate M
2. **Randomized Grover**: Try random numbers of iterations
3. **Fixed-point Grover**: Modified version that converges without oscillating

The randomized approach: if we pick k uniformly at random from [0, √N], we have ≥50% chance of success.
</details>

**Q3**: Can Grover's algorithm solve NP-complete problems efficiently?

<details>
<summary>Click for answer</summary>

Not really "efficiently" in the complexity theory sense:
- Classical brute force: $O(2^n)$
- Grover's: $O(2^{n/2})$

This is still exponential! It's a quadratic speedup (halving the exponent), not polynomial time.

However, it's a significant practical speedup. For example, AES-128 key search goes from $2^{128}$ to $2^{64}$ operations.
</details>

## Section 11: Summary & Applications

### Key Takeaways

1. **Oracle marks targets** with phase flip, not bit flip
2. **Diffuser reflects** about the mean (uniform superposition)
3. **Optimal iterations**: $\frac{\pi}{4}\sqrt{N/M}$ for M targets in N items
4. **Too many iterations** decreases success probability!
5. **Quadratic speedup** is proven optimal for unstructured search

### Applications

| Application | Classical | Quantum |
|-------------|-----------|--------|
| Database search | $O(N)$ | $O(\sqrt{N})$ |
| Symmetric key search | $O(2^n)$ | $O(2^{n/2})$ |
| SAT solving | $O(2^n)$ | $O(2^{n/2})$ |
| Optimization | $O(N)$ | $O(\sqrt{N})$ |
| Graph coloring | Exponential | √ faster |

### Practical Considerations

1. **Oracle implementation** can be expensive (many ancillas, deep circuits)
2. **Noise** reduces success probability significantly
3. **Unknown M** requires additional techniques
4. **Better for verification** than discovery (check if solution exists)

### Next Steps

- **[Module 7.8: Amplitude Estimation](Module-08-Amplitude-Estimation.md)**: QPE + Grover
- **[Module 5.x: QAOA](../04-Adiabatic-Annealing/Module-02-QAOA-Workshop.md)**: Variational alternative
- **Post-Quantum Cryptography**: Why AES-256 is recommended